In [ ]:
import pandas as pd
from pathlib import Path


In [ ]:
# Vector trimestral del deflactor nacional (base 2005)
deflactor_mexico = [
    100.00, 100.52, 101.25, 102.34, 103.11, 103.22, 104.13, 105.47,
    106.51, 106.68, 107.82, 109.43, 110.60, 111.64, 113.14, 115.02,
    116.38, 116.44, 117.15, 118.06, 119.92, 120.15, 120.91, 122.37,
    123.36, 123.68, 124.58, 126.11, 127.28, 127.89, 129.23, 131.02,
    132.22, 132.30, 133.22, 134.41, 135.61, 135.83, 136.91, 138.25,
    138.86, 138.76, 139.51, 140.24, 141.01, 141.13, 142.17, 143.94,
    146.12, 147.27, 148.87, 151.10, 153.07, 153.03, 154.51, 156.41,
    157.65, 158.12, 158.83, 159.87, 161.42, 161.27, 162.59, 164.84,
    166.92, 168.12, 170.36, 173.84, 176.63, 179.02, 182.26, 185.73,
    188.76, 188.80, 190.23, 192.51, 195.96, 196.86, 199.11, 201.76,
    203.68, 205.14
]

def construir_deflactor_dataframe(deflactor, periodo_inicio="2005Q1"):
    """Construye un DataFrame con year/quarter y el deflactor para el merge."""
    fechas = pd.period_range(start=periodo_inicio, periods=len(deflactor), freq="Q")
    df_def = pd.DataFrame({"deflactor": deflactor, "fecha_idx": fechas})
    df_def["year"] = df_def["fecha_idx"].dt.year
    df_def["quarter"] = df_def["fecha_idx"].dt.quarter
    return df_def[["year", "quarter", "deflactor"]]


In [ ]:
def deflactar_nacional(ruta_archivo, columnas_monetarias, deflactor, periodo_inicio="2005Q1"):
    """
    Lee un archivo de Datos Nacionales, agrega el deflactor y genera columnas def_ para cada variable monetaria.
    Guarda el resultado con el sufijo _deflactado y regresa el DataFrame y la ruta de salida.
    """
    df = pd.read_csv(ruta_archivo)
    df["year"] = df["year"].astype(int)
    df["quarter"] = df["quarter"].astype(int)

    df_deflactor = construir_deflactor_dataframe(deflactor, periodo_inicio)
    df_final = pd.merge(df, df_deflactor, on=["year", "quarter"], how="left")

    if df_final["deflactor"].isna().any():
        raise ValueError("Faltan valores del deflactor para algunos trimestres.")

    faltantes = [col for col in columnas_monetarias if col not in df_final.columns]
    if faltantes:
        raise ValueError(f"Las columnas no est?n en el archivo: {faltantes}")

    for col in columnas_monetarias:
        df_final[f"def_{col}"] = (df_final[col] / df_final["deflactor"]) * 100

    nombre_archivo = Path(ruta_archivo)
    archivo_salida = nombre_archivo.with_name(f"{nombre_archivo.stem}_deflactado.csv")
    df_final.to_csv(archivo_salida, index=False)
    return df_final, archivo_salida


In [ ]:
# Ejemplo de uso con las variables monetarias principales
archivo_nacional = "20251126_Nacional.csv"
variables_a_deflactar = [
    "ing_mensual_hombres",
    "ing_mensual_mujeres",
    "ing_hora_hombres",
    "ing_hora_mujeres",
]

df_deflactado, archivo_salida = deflactar_nacional(
    archivo_nacional,
    variables_a_deflactar,
    deflactor_mexico,
)

print(f"Archivo guardado en: {archivo_salida}")
df_deflactado.head()
